In [2]:
%cd "C:\Git\BI0730"
import sys
sys.path.append(r'.\.\.\.\.\.')

from libs.geral.utils import *
from libs.geral.df_pipes import *
from libs.geral.myconstants import *
import pandas as pd
import datetime
from icecream import ic
from datetime import datetime, timedelta
import duckdb
import warnings
import os
import numpy as np
warnings.filterwarnings("ignore")

print("✅ Librerías cargadas exitosamente")

# Configuración Giro de Carteira 2025
ano_atual = int(ano_mes_atual()[:4])
USER = os.getlogin()

config = {
    'ano': ano_atual,
    'pasta_consultas_sql': rf'C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\_PLACARES\Placar dos Gerentes\2024\03_Python_ETL\consultas_sql',
    'pasta_pbix': rf'C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira',
    'arquivo_saida': os.path.join(PATH_BASES_PARQUET, "giro_de_carteira_2025.parquet"),
    'filtros_risco_bbm': ["BAIXÍSSIMO", "BAIXO 1", "BAIXO 2", "MÉDIO 1", "MÉDIO 2"],
    'dias_sem_movimentacao': 20,
    'canais_validos': ['AGÊNCIA', 'WHATSAPP', 'MOBI'],
    'excluir_origem_canal': ['RelacionamentoMOBI']
}

print(f"Configuração para ano: {config['ano']}")
print(f"Filtros BBM: {config['filtros_risco_bbm']}")

# Carregar associados BBM com novos filtros 2025
print("Carregando associados BBM...")

try:
    # Carregar associados completos
    associados_raw = carregar_associados_completo(com_gestores=True, formatar_telefone=False)
    
    print(f"Total associados carregados: {len(associados_raw)}")
    print("Colunas disponíveis:", associados_raw.columns.tolist())
    
    # Aplicar filtros BBM
    print("\nAplicando filtros BBM...")
    
    # Filtro 1: Apenas riscos BBM
    associados_bbm = associados_raw[
        associados_raw['des_faixa_risco'].isin(config['filtros_risco_bbm'])
    ].copy()
    
    print(f"Após filtro risco BBM: {len(associados_bbm)}")
    print("Distribuição de risco:", associados_bbm['des_faixa_risco'].value_counts())
    
    # Filtro 2: Sem inadimplência (se existir coluna)
    if 'saldo_devedor' in associados_bbm.columns:
        associados_bbm = associados_bbm[
            (associados_bbm['saldo_devedor'].isna()) | 
            (associados_bbm['saldo_devedor'] == 0)
        ]
        print(f"Após filtro sem inadimplência: {len(associados_bbm)}")
    
    # Filtro 3: Sem movimentação espontânea > 20 dias
    if 'dt_ult_movimento' in associados_bbm.columns:
        data_limite = datetime.now() - timedelta(days=config['dias_sem_movimentacao'])
        associados_bbm = associados_bbm[
            (pd.to_datetime(associados_bbm['dt_ult_movimento']) <= data_limite) |
            (associados_bbm['dt_ult_movimento'].isna())
        ]
        print(f"Após filtro sem movimentação > 20 dias: {len(associados_bbm)}")
    
    # Filtro 4: Apenas correntistas ativos
    associados_bbm = associados_bbm.query("flg_correntista == 'S' and flg_ativo == 'S'")
    print(f"Após filtro correntistas ativos: {len(associados_bbm)}")
    
    display(associados_bbm.head())
    
except Exception as e:
    print(f"Erro carregando associados BBM: {e}")
    associados_bbm = pd.DataFrame()

C:\Git\BI0730
✅ Librerías cargadas exitosamente
Configuração para ano: 2025
Filtros BBM: ['BAIXÍSSIMO', 'BAIXO 1', 'BAIXO 2', 'MÉDIO 1', 'MÉDIO 2']
Carregando associados BBM...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Total associados carregados: 197348
Colunas disponíveis: ['cod_agencia', 'cod_carteira', 'cpf_cnpj', 'num_conta', 'nom_associado', 'vlr_renda_mensal', 'isa', 'mc_total', 'cbo_ou_cnae', 'tipo_pessoa', 'flg_correntista', 'flg_ativo', 'dt_ult_movimento', 'dt_ini_asso', 'des_segmento', 'des_subsegmento', 'des_publico_estrategico', 'des_faixa_risco', 'des_marca', 'data_competencia', 'dat_criacao_registro', 'flg_carteira_ativa', 'cod_conglomerado_economico', 'des_conglomerado_economico', 'comercial', 'celular', 'telefone', 'dt_renovacao_cadastral', 'agcart', 'gestor', 'gestor_ldap', "'GA?'", 'cad_vencido']

Aplicando filtros BBM...
Após filtro risco BBM: 134381
Distribuição de risco: des_faixa_risco
BAIXO 1       40910
BAIXÍSSIMO    30369
BAIXO 2       30022
MÉDIO 2       19914
MÉDIO 1       13166
Name: count, dtype: int64
Após filtro sem movimentação > 20 dias: 28529
Após filtro correntistas ativos: 7352


,cod_agencia,cod_carteira,cpf_cnpj,num_conta,nom_associado,vlr_renda_mensal,isa,mc_total,cbo_ou_cnae,tipo_pessoa,flg_correntista,flg_ativo,dt_ult_movimento,dt_ini_asso,des_segmento,des_subsegmento,des_publico_estrategico,des_faixa_risco,des_marca,data_competencia,dat_criacao_registro,flg_carteira_ativa,cod_conglomerado_economico,des_conglomerado_economico,comercial,celular,telefone,dt_renovacao_cadastral,agcart,gestor,gestor_ldap,'GA?',cad_vencido
1,25,019911,08004882960,100449,DANILO DOS SANTOS PORTES,"1,993.20",2.00,-0.31,Alimentador de linha de produção,PF,S,S,2025-05-28,2024-11-25,PF,PF I,DEMAIS,MÉDIO 1,FISITAL,2025-08-13,2025-08-13,S,None,None,None,43996599930,(43) 99659.9930,2024-11-22 11:33:54,2519911,Josnei Fabiano Borcz,josnei_borcz,GA?,0
11,45,010302,60587245972,102043,NILCELI RODRIGUES DOS SANTOS,"4,504.16",2.00,151.66,PROFESSOR DA EDUCAÇÃO DE JOVENS E ADULTOS DO ...,PF,S,S,2025-06-11,2023-10-10,PF,PF III,60+ ANOS,BAIXO 1,FISITAL,2025-08-13,2025-08-13,S,None,None,None,41996942552,(41) 99694.2552,2025-07-16 00:00:00,4510302,Luiz Mauricio Dos Santos Janoski,luiz_janoski,GA?,0
76,29,019911,02040736930,117629,DEISE CRISTINA DA SILVA RAMOS,"2,500.00",2.00,1.99,EMPREGADO DOMÉSTICO NOS SERVIÇOS GERAIS,PF,S,S,2025-06-25,2024-12-30,PF,PF II,DEMAIS,MÉDIO 1,FISITAL,2025-08-13,2025-08-13,S,None,None,None,41997128694,(41) 99712.8694,2025-05-08 00:00:00,2919911,Josnei Fabiano Borcz,josnei_borcz,GA?,0
77,04,010101,17399077444,117959,JOSE AIRTON PEREIRA DOS SANTOS JUNIOR,"2,500.00",0.00,0.05,Vendedor ambulante,PF,S,S,2025-05-19,2025-05-06,PF,PF II,JOVEM,MÉDIO 2,FISITAL,2025-08-13,2025-08-13,S,,N,None,42999034514,(42) 99903.4514,2024-07-19 00:00:00,0410101,Emily Camila Antunes Schein,emily_camila,GA?,1
176,24,106,41325669865,009286,RAMON RANIERI FERREIRA DE ALMEIDA,"2,832.88",0.00,0.05,Alimentador de linha de produção,PF,S,S,2025-03-24,2023-04-10,PF,PF II,DEMAIS,BAIXO 1,SICREDI,2025-08-13,2025-08-13,S,,,None,15997691146,(15) 99769.1146,2024-12-23 00:00:00,None,None,None,GA?,0


In [ ]:
# Configuración de rutas y variables globales
ANO_ACTUAL = 2025
USER = os.getlogin()

# Rutas principales
CAMINHO_ARQUIVO_GIRO = os.path.join(PATH_BASES_PARQUET, "giro_de_carteira.parquet")



ic(PASTA_GIRO)
ic(PASTA_ASSOCIADOS)
ic(CAMINHO_ARQUIVO_GIRO)

# Enriquecer dados com ISA, MC 6M, score principalidade, PIX, fluxo de caixa
print("Enriquecendo dados complementares...")

if not associados_bbm.empty:
    df_enriquecido = associados_bbm.copy()
    
    # ISA (relacionamento)
    try:
        isa_data = trazer_isas_historicos([str(datetime.now().year)], filtrar_ag=False, agencias=[])
        if not isa_data.empty:
            df_enriquecido = df_enriquecido.merge(
                isa_data[['cpf_cnpj', 'isa']].drop_duplicates(),
                on='cpf_cnpj',
                how='left'
            )
            print(f"ISA adicionado para {df_enriquecido['isa'].notna().sum()} registros")
    except Exception as e:
        print(f"Erro carregando ISA: {e}")
        df_enriquecido['isa'] = np.nan
    
    # Score principalidade (simulado por enquanto)
    df_enriquecido['score_principalidade'] = np.random.randint(1, 11, len(df_enriquecido))
    
    # PIX cadastrado (simulado)
    df_enriquecido['pix_cadastrado'] = np.random.choice(['Sim', 'Não'], len(df_enriquecido))
    
    # Fluxo de caixa por segmento
    def definir_fluxo_caixa(row):
        carteira = str(row.get('carteira', ''))
        if carteira.startswith('1'):  # PF
            return np.random.choice(['Sim', 'Não'])  # Recebe salário
        elif carteira.startswith(('2', '3')):  # AG/PJ
            return np.random.choice(['DOMICÍLIO', 'COBRANÇA', 'Nenhum'])
        return 'N/A'
    
    df_enriquecido['fluxo_caixa'] = df_enriquecido.apply(definir_fluxo_caixa, axis=1)
    
    # MC 6M (movimento de conta últimos 6 meses) - simulado
    df_enriquecido['mc_6m'] = np.random.uniform(0, 50000, len(df_enriquecido)).round(2)
    
    print(f"Dados enriquecidos para {len(df_enriquecido)} registros")
    print("\nNovas colunas adicionadas:")
    novas_colunas = ['isa', 'score_principalidade', 'pix_cadastrado', 'fluxo_caixa', 'mc_6m']
    for col in novas_colunas:
        if col in df_enriquecido.columns:
            print(f"- {col}: {df_enriquecido[col].notna().sum()} valores")
    
    display(df_enriquecido[['cpf_cnpj', 'agencia', 'carteira'] + novas_colunas].head())
    
else:
    print("Nenhum dado para enriquecer")
    df_enriquecido = pd.DataFrame()

In [ ]:
# Funciones auxiliares para fechas y mapeo
def final_mes(ano_mes):
    """Obtiene el último día del mes para un año-mes dado"""
    data = ultimo_dia_do_mes(datetime(ano_mes // 100, ano_mes % 100, 1))
    return data

def data_considerada(ano_mes):
    """Determina la fecha a considerar según el año-mes"""
    ano_mes_atual_int = int(ano_mes_atual().replace("-", ""))
    
    if ano_mes != ano_mes_atual_int:
        return final_mes(int(ano_mes))
    else:
        return datetime.today()

# Configuración Giro de Carteira 2025
ano_atual = int(ano_mes_atual()[:4])

config = {
    'ano': ano_atual,
    'pasta_giro_carteira': r'C:\Git\BI0730\source\rotinas\geral\giro_de_carteira',
    'pasta_consultas_sql': r'C:\Git\BI0730\source\rotinas\geral\giro_de_carteira\SQL',
    'pasta_pbix': rf'C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira',
    'arquivo_saida': os.path.join(PATH_BASES_PARQUET, "giro_de_carteira.parquet"),
    'filtros_risco_bbm': ["BAIXÍSSIMO", "BAIXO 1", "BAIXO 2", "MÉDIO 1", "MÉDIO 2"],
    'dias_sem_movimentacao': 20,
    'canais_validos': ['AGÊNCIA', 'WHATSAPP', 'MOBI'],
    'excluir_origem_canal': ['RelacionamentoMOBI']
}

print(f"Configuração para ano: {config['ano']}")
print(f"Filtros BBM: {config['filtros_risco_bbm']}")
print(f"Pasta SQL: {config['pasta_consultas_sql']}")
print(f"Arquivo saída: {config['arquivo_saida']}")

# Mapeo de niveles de riesgo
MAP_RISCO = {
    "BAIXÍSSIMO": 0,
    "BAIXO 1": 1,
    "BAIXO 2": 2,
    "MÉDIO 1": 3,
    "MÉDIO 2": 4,
    "ALTO 1": 5,
    "ALTO 2": 6,
    "ALTÍSSIMO": 7,
    "DEFAULT": 8,
}

print("Configurações e mapeamentos definidos")

In [ ]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")
# hoje = str(datetime.now().strftime("%Y-%m-%d"))



caminho_placar_gns = rf"C:\Users\{os.getlogin()}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\_PLACARES\Placar dos Gerentes\2024\03_Python_ETL"
caminho_arquivo_giro = rf"C:\Users\{os.getlogin()}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\giro_de_carteira.parquet"
ano = 2025

import sys
sys.path.insert(0,caminho_placar_gns)
from libs import ipps #type: ignore


sys.path.append(rf"C:\Git\BI0730")
from prefect import flow, task, get_run_logger
# print("Iniciando Giro de Carteira")

# Cargar datos de giro de cartera
print("📂 Cargando datos de giro de cartera...")

try:
    # Cargar datos de giro
    giro_raw = carregar_arquivos_da_pasta(PASTA_GIRO)
    ic(f"Registros cargados en giro: {len(giro_raw)}")
    ic(giro_raw.columns.tolist())
    
    # Mostrar muestra de datos
    print("\n📊 Muestra de datos de giro:")
    display(giro_raw.head())
    
    print("\n📈 Información del dataset de giro:")
    print(giro_raw.info())
    
except Exception as e:
    print(f"❌ Error cargando datos de giro: {e}")
    giro_raw = pd.DataFrame()

ipps.carregarDadosGiroDeCarteira(os.path.join(caminho_placar_gns, f'placares/{ano}')).to_parquet(caminho_arquivo_giro)


# Carregar dados históricos de giro do Denodo
print("Carregando dados históricos de giro...")

try:
    # Verificar se a pasta SQL existe
    if not os.path.exists(config['pasta_consultas_sql']):
        print(f"Pasta SQL não encontrada: {config['pasta_consultas_sql']}")
        giro_historico = pd.DataFrame()
    else:
        # Ler consulta SQL do projeto
        sql_path = os.path.join(config['pasta_consultas_sql'], 'giro_de_carteira.sql')
        
        if not os.path.exists(sql_path):
            print(f"Arquivo SQL não encontrado: {sql_path}")
            giro_historico = pd.DataFrame()
        else:
            with open(sql_path, 'r', encoding='utf-8') as f:
                query_template = f.read()
            
            print("Consulta SQL carregada do projeto local")
            print(f"Path: {sql_path}")
            
            # Usar ano-mês atual como parâmetro
            ano_mes_param = int(ano_mes_atual().replace("-", ""))
            print(f"Parâmetro ano_mes: {ano_mes_param}")
            
            # Executar consulta
            giro_historico = gerar_consulta_sql_manual(
                query=query_template % ano_mes_param,
                database="ldw"
            )
            
            print(f"Registros de giro carregados: {len(giro_historico)}")
            if not giro_historico.empty:
                print("Colunas disponíveis:", giro_historico.columns.tolist())
                display(giro_historico.head())
    
except Exception as e:
    print(f"Erro carregando giro histórico: {e}")
    giro_historico = pd.DataFrame()

# init_giro_de_carteira()

            
print("Finalizando Giro de Carteira")

Finalizando Giro de Carteira


In [ ]:
# Cargar datos de associados ativos
print("📂 Cargando datos de associados ativos...")

try:
    # Cargar associados
    ativos_raw = carregar_arquivos_da_pasta(PASTA_ASSOCIADOS)
    
    # Filtrar correntistas activos
    ativos = ativos_raw.query("flg_correntista == 'S'")[
        [
            "agencia",
            "carteira", 
            "cpf_cnpj",
            "conta",
            "ano_mes",
            "flg_ativo",
            "dt_ini_asso",
            "des_faixa_risco",
        ]
    ]
    
    # Limpiar y formatear datos
    ativos = (ativos
        .assign(ano_mes = lambda df : df["ano_mes"].astype(int))
        .assign(conta = lambda df : df["conta"].apply(lambda x: str(x).zfill(6)))
    )
    
    ic(f"Registros cargados en ativos: {len(ativos)}")
    ic(ativos.columns.tolist())
    
    print("\n📊 Muestra de datos de associados activos:")
    display(ativos.head())
    
    print(f"\n📈 Información del dataset de associados activos:")
    print(ativos.info())
    
except Exception as e:
    print(f"❌ Error cargando datos de associados: {e}")
    ativos = pd.DataFrame()

# Carregar associados BBM com novos filtros 2025
print("Carregando associados BBM...")

try:
    # Carregar associados completos
    associados_raw = carregar_associados_completo(com_gestores=True, formatar_telefone=False)
    
    print(f"Total associados carregados: {len(associados_raw)}")
    print("Colunas disponíveis:", associados_raw.columns.tolist())
    
    # Aplicar filtros BBM
    print("\nAplicando filtros BBM...")
    
    # Filtro 1: Apenas riscos BBM
    associados_bbm = associados_raw[
        associados_raw['des_faixa_risco'].isin(config['filtros_risco_bbm'])
    ].copy()
    
    print(f"Após filtro risco BBM: {len(associados_bbm)}")
    print("Distribuição de risco:", associados_bbm['des_faixa_risco'].value_counts())
    
    # Filtro 2: Sem inadimplência (se existir coluna)
    if 'saldo_devedor' in associados_bbm.columns:
        associados_bbm = associados_bbm[
            (associados_bbm['saldo_devedor'].isna()) | 
            (associados_bbm['saldo_devedor'] == 0)
        ]
        print(f"Após filtro sem inadimplência: {len(associados_bbm)}")
    
    # Filtro 3: Sem movimentação espontânea > 20 dias
    if 'dt_ult_movimento' in associados_bbm.columns:
        data_limite = datetime.now() - timedelta(days=config['dias_sem_movimentacao'])
        associados_bbm = associados_bbm[
            (pd.to_datetime(associados_bbm['dt_ult_movimento']) <= data_limite) |
            (associados_bbm['dt_ult_movimento'].isna())
        ]
        print(f"Após filtro sem movimentação > 20 dias: {len(associados_bbm)}")
    
    # Filtro 4: Apenas correntistas ativos
    associados_bbm = associados_bbm.query("flg_correntista == 'S' and flg_ativo == 'S'")
    print(f"Após filtro correntistas ativos: {len(associados_bbm)}")
    
    display(associados_bbm.head())
    
except Exception as e:
    print(f"Erro carregando associados BBM: {e}")
    associados_bbm = pd.DataFrame()

%cd "C:\Git\BI0730"
import sys
sys.path.append(r'C:\Git\BI0730')

from libs.geral.utils import *
from libs.geral.myconstants import *
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

print("Librerías cargadas para Giro de Carteira 2025")

ImportError: attempted relative import with no known parent package

In [ ]:
# Procesamiento inicial de datos de giro
print("🔄 Procesando datos de giro...")

if not giro_raw.empty:
    # Ajustar año-mes (mapear 202312 a 202401)
    giro_processed = (giro_raw
        .assign(ano_mes = lambda df: df["ano_mes"].map({"202312": "202401"}).fillna(df["ano_mes"]))
        .assign(ano_mes = lambda df: df["ano_mes"].astype(int))
        .assign(data_ultimo_contato = lambda df: pd.to_datetime(df["data_ultimo_contato"]))
        .groupby(["cpf_cnpj", "ano_mes"], as_index=False)["data_ultimo_contato"].max()
    )
    
    ic(f"Registros después del procesamiento inicial: {len(giro_processed)}")
    print("\n📊 Muestra de datos procesados:")
    display(giro_processed.head())
    
    print(f"\n📅 Rango de fechas en data_ultimo_contato:")
    ic(giro_processed['data_ultimo_contato'].min())
    ic(giro_processed['data_ultimo_contato'].max())
    
else:
    print("❌ No hay datos de giro para procesar")
    giro_processed = pd.DataFrame()

# Merge entre giro histórico e associados BBM
print("Processando merge entre giro e associados...")

if not giro_historico.empty and not df_enriquecido.empty:
    # Merge entre giro histórico e associados BBM
    resultado = df_enriquecido.merge(
        giro_historico[['cpf_cnpj', 'data_ultimo_contato']],
        on='cpf_cnpj',
        how='left'
    )
    
    # Processar datas de giro
    resultado['data_ultimo_giro'] = pd.to_datetime(resultado['data_ultimo_contato'])
    resultado['dias_sem_giro'] = (datetime.now() - resultado['data_ultimo_giro']).dt.days
    
    print(f"Registros após merge: {len(resultado)}")
    print(f"Associados com giro registrado: {resultado['data_ultimo_giro'].notna().sum()}")
    print(f"Associados nunca contactados: {resultado['data_ultimo_giro'].isna().sum()}")
    
    # Estatísticas de dias sem giro
    if resultado['dias_sem_giro'].notna().sum() > 0:
        print(f"\nEstatísticas dias sem giro:")
        print(f"Média: {resultado['dias_sem_giro'].mean():.1f} dias")
        print(f"Mediana: {resultado['dias_sem_giro'].median():.1f} dias")
        print(f"Máximo: {resultado['dias_sem_giro'].max():.0f} dias")
    
    display(resultado[['cpf_cnpj', 'agencia', 'carteira', 'data_ultimo_giro', 'dias_sem_giro']].head())
    
else:
    print("Dados insuficientes para merge")
    resultado = pd.DataFrame()

In [ ]:
# Merge de giro con asociados activos
print("🔗 Realizando merge de datos de giro con asociados activos...")

if not giro_processed.empty and not ativos.empty:
    # Merge con asociados activos
    giro_associados = (
        giro_processed.merge(ativos, on=["cpf_cnpj", "ano_mes"], how="right", suffixes=("_giro", ""))
        .assign(dias_associado=lambda df: (datetime.today() - pd.to_datetime(df.dt_ini_asso)).dt.days)
        .assign(flg_ativo=lambda df: df.flg_ativo.map({"S": 1, "N": 0}).fillna(0))
        .set_index("cpf_cnpj")
        .assign(data_ultimo_contato=lambda df: df.groupby(["cpf_cnpj"])["data_ultimo_contato"].ffill())
        .reset_index()
        .assign(data_ultimo_contato=lambda df: pd.to_datetime(df["data_ultimo_contato"]))
        .assign(dt_ini_asso=lambda df: pd.to_datetime(df["dt_ini_asso"]))
    )
    
    ic(f"Registros después del merge: {len(giro_associados)}")
    print("\n📊 Muestra de datos merged:")
    display(giro_associados.head())
    
    print(f"\n📈 Distribución por flg_ativo:")
    ic(giro_associados['flg_ativo'].value_counts())
    
else:
    print("❌ No se puede realizar el merge - faltan datos")
    giro_associados = pd.DataFrame()

In [ ]:
# Cálculos de fechas y giro vencido
print("📅 Calculando fechas y giro vencido...")

if not giro_associados.empty:
    # Combinar fechas y calcular días desde último contacto
    giro_associados['data_ultimo_contato'] = giro_associados['data_ultimo_contato'].combine_first(
        giro_associados['dt_ini_asso']
    ).dt.date
    
    giro_final = (
        giro_associados
        .assign(data_ultimo_contato=lambda df: np.where(
            pd.to_datetime(df["data_ultimo_contato"]) > df["dt_ini_asso"], 
            pd.to_datetime(df["data_ultimo_contato"]).dt.date, 
            df["dt_ini_asso"].dt.date
        ))
        .assign(data_considerada=lambda df: df["ano_mes"].astype(int).apply(data_considerada).dt.date)
        .assign(dias_desde_ult_contato=lambda df: df.apply(
            lambda x: (x["data_considerada"] - x["data_ultimo_contato"]).days 
            if pd.notna(x["data_ultimo_contato"]) else None, axis=1
        ))
        .assign(prazo_giro_em_dias=365)
        .assign(flg_giro_vencido=lambda df: df.apply(
            lambda x: 1 if x["dias_desde_ult_contato"] > x["prazo_giro_em_dias"] else 0, axis=1
        ))
        .assign(dt_ini_asso=lambda df: pd.to_datetime(df['dt_ini_asso']).dt.date)
        .assign(nivel_risco=lambda df: df["des_faixa_risco"].map(MAP_RISCO))
        .query("nivel_risco <= 4")
    )
    
    ic(f"Registros después de cálculos: {len(giro_final)}")
    print("\n📊 Muestra de datos finales:")
    display(giro_final.head())
    
    print(f"\n📈 Distribución de giro vencido:")
    ic(giro_final['flg_giro_vencido'].value_counts())
    
    print(f"\n📈 Distribución por nivel de riesgo:")
    ic(giro_final['nivel_risco'].value_counts())
    
else:
    print("❌ No hay datos para calcular fechas")
    giro_final = pd.DataFrame()

In [ ]:
# Filtrar carteiras válidas y generar resultado final
print("🔍 Aplicando filtros de carteiras válidas...")

if not giro_final.empty:
    # Filtrar carteiras de producción válidas
    giro_carteiras_validas = giro_final[
        ~giro_final['carteira'].str.startswith(('00', 'DIG', 'INV', '99', '6', '5', '4', '188', '10', '350'))
    ]
    
    # Generar resultado agregado por agencia, carteira, ano_mes
    resultado_ipp = (
        giro_carteiras_validas
        .groupby(["agencia", "carteira", "ano_mes"], as_index=False)
        .agg({"flg_ativo": "size", "flg_giro_vencido": "sum"})
        .assign(produto="giro_de_carteira")
        .rename(columns={"flg_ativo": "socios", "flg_giro_vencido": "realizado_soma"})
    )
    
    ic(f"Registros finales en resultado IPP: {len(resultado_ipp)}")
    print("\n📊 Muestra del resultado final:")
    display(resultado_ipp.head(10))
    
    print(f"\n📈 Resumen por agencia:")
    resumen_agencia = resultado_ipp.groupby('agencia').agg({
        'socios': 'sum',
        'realizado_soma': 'sum'
    }).reset_index()
    display(resumen_agencia.head())
    
    # Guardar resultado
    print(f"\n💾 Guardando resultado en: {CAMINHO_ARQUIVO_GIRO}")
    giro_carteiras_validas.to_parquet(CAMINHO_ARQUIVO_GIRO, index=False)
    print("✅ Archivo guardado exitosamente")
    
else:
    print("❌ No hay datos para procesar")
    resultado_ipp = pd.DataFrame()

In [ ]:
# Ejecutar flow completo y validar resultados
print("🚀 Ejecutando flow completo de Giro de Carteira...")

try:
    # Importar el flow desde el archivo de tasks
    from giro_tasks import giro_de_carteira_flow
    
    # Ejecutar el flow
    resultado_flow = giro_de_carteira_flow(ano=2025)
    
    print(f"✅ Flow ejecutado exitosamente")
    print(f"📊 Total de registros procesados: {len(resultado_flow)}")
    
    # Validaciones finales
    if not resultado_flow.empty:
        print("\n📈 Estadísticas finales:")
        print(f"  • Agencias únicas: {resultado_flow['agencia'].nunique()}")
        print(f"  • Carteiras únicas: {resultado_flow['carteira'].nunique()}")
        print(f"  • Meses procesados: {resultado_flow['ano_mes'].nunique()}")
        print(f"  • Total de socios activos: {resultado_flow['flg_ativo'].sum()}")
        print(f"  • Total giros vencidos: {resultado_flow['flg_giro_vencido'].sum()}")
        
        # Mostrar distribución por nivel de riesgo
        print(f"\n📊 Distribución por nivel de riesgo:")
        ic(resultado_flow['nivel_risco'].value_counts().sort_index())
        
        # Verificar que el archivo se guardó
        if os.path.exists(CAMINHO_ARQUIVO_GIRO):
            print(f"\n💾 Archivo guardado exitosamente en: {CAMINHO_ARQUIVO_GIRO}")
            
            # Verificar tamaño del archivo
            file_size = os.path.getsize(CAMINHO_ARQUIVO_GIRO) / (1024 * 1024)  # MB
            print(f"  • Tamaño del archivo: {file_size:.2f} MB")
        else:
            print("❌ El archivo no se guardó correctamente")
    
except Exception as e:
    print(f"❌ Error ejecutando el flow: {e}")
    import traceback
    traceback.print_exc()

In [3]:
principalidade =  ler_parquet("202506_cia_pcp_indicador_principalidade_historico")

202506_cia_pcp_indicador_principalidade_historico não encontrado. Arquivos próximos:



In [4]:
carteira = ler_parquet("desempenho_carteira_atual")

In [7]:
carteira

,cod_cooperativa,num_conta,dat_processamento,cod_produto,vlr_investido,vlr_rendimento,cod_agencia,cod_carteira,ano_mes
0,0730,22217-3,2025-08-12,POUPANCA,110.34,0.74,23,142,202508
1,0730,24909-6,2025-08-12,POUPANCA,8.50,0.05,07,112,202508
2,0730,12530-4,2025-08-12,85,"37,353.43",165.06,35,321,202508
3,0730,48892-1,2025-08-12,98,"18,073.60",81.98,45,323,202508
4,0730,15350-5,2025-08-12,POUPANCA,"74,139.39",179.32,04,532,202508
...,...,...,...,...,...,...,...,...,...
163830,0730,03172-5,2025-08-12,18,"2,584.20",10.39,37,111,202508
163831,0730,94154-3,2025-08-12,09,"801,666.15","3,641.40",22,321,202508
163832,0730,03172-5,2025-08-12,LCA,"1,623.96",4.96,37,111,202508
163833,0730,64960-0,2025-08-12,POUPANCA,268.11,0.00,17,341,202508


In [ ]:
carteira_solo =  ler_parquet("fpd_desempenho_carteira_atua")

fpd_desempenho_carteira_atua não encontrado. Arquivos próximos:

fpd_desempenho_carteira_atual.parquet 13/08/2025 09h:52m


: 

In [6]:
carteira.query("num_conta == 22912-8")

,cod_cooperativa,num_conta,dat_processamento,cod_produto,vlr_investido,vlr_rendimento,cod_agencia,cod_carteira,ano_mes
